## Uninformed Search
Uninformed search algorithms such as breadth-first search (BFS) and depth-first search (DFS) do not attempt to apply intelligence to the  direction of the search. For this reason they are often called "brute-force" search methods. While both algorithms are uninformed, their differing implementations make them suitable for different problems:
+ **Breadth-first search**: Given a search position at a particular state, BFS examines all of that state's successors before searching any farther. Think of it as a layer-by-layer approach: BFS searches all the successors, then all the successors of the successors, then the successors of the successors of the successors, etc., until it finds a goal state. The algorithm is both complete (it is guaranteed to find a goal node in a finite state space) and optimal (it guarantees that no goal state in the state space will be closer to the initial state than the goal state that it finds).
+ **Depth-first search**: Given a search position at a particular node, DFS examines just one successor while storing the others in a stack; then examines one successor of the successor while storing the remainder of the successor's successors in the stack; and so forth. Essentially, it probes a potentially complete path until it either finds a goal state or deems it a blind alley; then it backs up a step and continues, performing the same steps. Like BFS, the algorithm runs until it finds a goal state and is complete. However, DFS is not optimal; it may bypass an easily reachable goal state and find a distant one instead.

### Using Uninformed Search on the N-Rooks Puzzle 
In the N-Rooks puzzle you must place N rooks on a NxN chess board in such a way that no rooks attack each other. The basic elements of the search can be defined as follows:
+ State space: the set of all possible configurations of 0 through N rooks on a NxN board. The size of this space increases exponentially with N, as it is equal to (N<sup>2</sup>)(N<sup>2</sup> - 1)(N<sup>2</sup> - 2) ... (N<sup>2</sup> - N).
+ Initial state: an empty board
+ Goal state: a NxN board with exactly one rook on each row and exactly one rook on each column. The problem has many goal states.
+ Successor function: the set of NxN boards with one more rook on the board than the predecessor state. 

It seems likely that DFS will be more suitable than BFS to solve the N-Rooks puzzle. DFS can start exploring possible solutions with N rooks almost immediately, after traversing just N-1 states. Meanwhile, BFS must fully investigate N<sup>N-1</sup> states before it can reach a state that might be a goal state.

Let's take a look at the code supplied by Professor David Crandall and instructor Zehua Zhang.

In [2]:
import sys
from time import clock
from collections import deque

# Count # of pieces in given row
def count_on_row(board, row):
    return sum( board[row] ) 

# Count # of pieces in given column
def count_on_col(board, col):
    return sum( [ row[col] for row in board ] ) 

# Count total # of pieces on board
def count_pieces(board):
    return sum([ sum(row) for row in board ] )

# Return a string with the board rendered in a human-friendly format
def printable_board(board):
    return "\n".join([ " ".join([ "R" if col else "_" for col in row ]) for row in board])

# Add a piece to the board at the given position, and return a new board (doesn't change original)
def add_piece(board, row, col):
    return board[0:row] + [board[row][0:col] + [1,] + board[row][col+1:]] + board[row+1:]


#### Data Structures
Note that in the instructors' code, the state is held in a list of lists -- a very inefficient data structure. In my next search algorithm notebook, I will show how to use numpy arrays to good effect in a greedy best-first search.

In the `solve()` function (below), I use a `collections.deque` as a container for the fringe. It can act as either a LIFO stack or a FIFO queue depending on the function used to remove an element.

#### Optimizing the Successor Function
My `successors2()` function contains the following optimizations:
+Successors are generated from only a single column, the leftmost empty column. If no empty column exists, then the function returns an empty list, indicating the absence of successors.
+Successors are not generated from rows that are already occupied in the predecessor node.

#### Functional Programming
The `solve()` function accepts a parameter removefunc which is used as a function to remove states from the fringe. When `deque.pop` is passed as an argument, the `solve()` function implements DFS because the deque acts as LIFO queue (i.e., a stack). Passing `deque.popleft` as an argument makes the `solve()` function implement BFS because the deque acts as a FIFO queue. 

#### Performance Testing Code
In order to test the time efficiency of BFS vs. DFS I created a function `timeSolution()` that measures the wall clock elapsed time needed to find a goal state.

The code in the cell below is my own work.

In [3]:
def leftmost_empty_column(board):
    '''
    returns index of leftmost empty column on board. Lower bound = 0, upper bound = N - 1
    returns -1 if there are no empty columns
    '''
    for i in range(N):
        if count_on_col(board, i) == 0:
            return i
    return -1

def empty_rows(board):
    '''
    Returns a list of row indexes for the rows already occupied by rooks
    '''
    return [r for r in range(N) if count_on_row(board, r) == 0]

# Get list of successors of given board state
def successors2(board):
    '''
    Modifications from instructor-supplied code:
    * No successors returned for occupied columns. Algorithm moves from left to right. 
      Successors are only selected from the leftmost empty column
    * No successors returned for already-occupied rows
    * Returns empty list if state has no successors (i.e., no empty columns remain on board)
    '''
    c = leftmost_empty_column(board)
    if c == -1:
        return []
    empty = empty_rows(board)
    return [add_piece(board, r, c) for r in empty]

# check if board is a goal state
def is_goal(board):
    return count_pieces(board) == N and \
        all( [ count_on_row(board, r) <= 1 for r in range(0, N) ] ) and \
        all( [ count_on_col(board, c) <= 1 for c in range(0, N) ] )

# Solve n-rooks!
def solve(initial_board, removeFunc):
    '''
    this started as the instructor-supplied depth-first search code. 
    Changes by Chris Falter:
        * call successors2 func, rather than the original successors func
        * Use deque instead of list
        * implemented as a general solver for both BFS and DFS. The algorithm is determined
          by the choice of removeFunc. If removal is from right, fringe is treated as a
          stack, so it's DFS. If removal is from left, fringe is treated as a FIFO queue,
          so it's BFS.
    '''
    fringe = deque([initial_board])
    while len(fringe) > 0:        
        for s in successors2(removeFunc(fringe)):
            if is_goal(s):
                return(s)
            fringe.append(s)
    return False

def solveDFS(initial_board):
    '''
    implements depth-first search for nrooks problem
    '''
    return solve(initial_board, deque.pop)

def solveBFS(initial_board):
    '''
    implements breadth-first search for nroooks problem
    '''
    return solve(initial_board, deque.popleft)

def timeSolution(solveFunc, initial_board, algorithm):
    print ("using " + algorithm)
    start = clock()
    solution = solveFunc(initial_board)
    elapsed = clock() - start
    print (printable_board(solution) if solution else "Sorry, no solution found. :(")
    print ("solved in " + str(elapsed) + " seconds\n\n")   


### Run the Code
To test the algorithms' time efficiency, configure the dimensions of the chess board and the number of rooks by setting the value of N as desired in the cell below. Then run the cell to see what happens!

In [4]:
# This is N, the size of the board.
N = 10

# The board is stored as a list-of-lists. Each inner list is a row of the board.
# A zero in a given square indicates no piece, and a 1 indicates a piece.
initial_board = [[0]*N]*N
print ("Starting from initial board:\n" + printable_board(initial_board) + "\n\nLooking for solutions...\n")
timeSolution(solveDFS, initial_board, "DFS")
timeSolution(solveBFS, initial_board, "BFS")

Starting from initial board:
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _

Looking for solutions...

using DFS
_ _ _ _ _ _ _ _ _ R
_ _ _ _ _ _ _ _ R _
_ _ _ _ _ _ _ R _ _
_ _ _ _ _ _ R _ _ _
_ _ _ _ _ R _ _ _ _
_ _ _ _ R _ _ _ _ _
_ _ _ R _ _ _ _ _ _
_ _ R _ _ _ _ _ _ _
_ R _ _ _ _ _ _ _ _
R _ _ _ _ _ _ _ _ _
solved in 0.0003965890061093592 seconds


using BFS
R _ _ _ _ _ _ _ _ _
_ R _ _ _ _ _ _ _ _
_ _ R _ _ _ _ _ _ _
_ _ _ R _ _ _ _ _ _
_ _ _ _ R _ _ _ _ _
_ _ _ _ _ R _ _ _ _
_ _ _ _ _ _ R _ _ _
_ _ _ _ _ _ _ R _ _
_ _ _ _ _ _ _ _ R _
_ _ _ _ _ _ _ _ _ R
solved in 74.30493137491169 seconds




DFS wins by several orders of magnitude! This is not surprising, considering that it has much lower time complexity than BFS, as I explained previously. It also has far lower memory complexity; the DFS fringe will hold a maximum of `2N - 1` states, while the BFS fringe will hold a maximum of `N!` states when it begins searching the final layer. With a 10 x 10 board, that amounts to 3,628,800 states in the fringe. 